In [21]:
import pandas as pd 

# Load the CSV file
df = pd.read_csv('/Users/jameswanwichet/VScode/Projects/Dataset/Data Sep1-Oct7.csv')

# Function to get all combinations of 5-day spans
def get_day_spans():
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    return [days[i:i+5] for i in range(len(days)-4)]

# Convert the 'วันที่' (date) column to a proper datetime format
df['วันที่'] = pd.to_datetime(df['วันที่'], format='%d-%m-%Y %H:%M')

# Extract day of the week (Monday-Sunday) and hour
df['DayOfWeek'] = df['วันที่'].dt.day_name()
df['Hour'] = df['วันที่'].dt.hour

# Clean sales column by removing commas and converting to numeric
df['ยอดขาย (ยืนยันแล้ว) (THB)'] = df['ยอดขาย (ยืนยันแล้ว) (THB)'].replace(',', '', regex=True).astype(float)

# Clean percentage columns and convert to float
df['อัตราการแวะเข้าชมสินค้า'] = df['อัตราการแวะเข้าชมสินค้า'].str.replace('%', '').astype(float) / 100
df['อัตราการสั่งซื้อ (ที่มีการสั่งซื้อทั้งหมด)'] = df['อัตราการสั่งซื้อ (ที่มีการสั่งซื้อทั้งหมด)'].str.replace('%', '').astype(float) / 100
df['อัตราการสั่งซื้อ (ยืนยันคำสั่งซื้อแล้ว)'] = df['อัตราการสั่งซื้อ (ยืนยันคำสั่งซื้อแล้ว)'].str.replace('%', '').astype(float) / 100

# Group by Day of the week and Hour to aggregate all the key indicators
df_grouped = df.groupby(['DayOfWeek', 'Hour']).agg({
    'ผู้เข้าชมสินค้า': 'sum',
    'ยอดเข้าชมสินค้า': 'sum',
    'สินค้าที่ถูกเข้าชม': 'sum',
    'ยอดขาย (ยืนยันแล้ว) (THB)': 'sum',
    'จำนวนที่ขายได้ (ยืนยันแล้ว)': 'sum',
    'อัตราการแวะเข้าชมสินค้า': 'mean',  # Use average for rates
    'อัตราการสั่งซื้อ (ที่มีการสั่งซื้อทั้งหมด)': 'mean',
    'อัตราการสั่งซื้อ (ยืนยันคำสั่งซื้อแล้ว)': 'mean'
}).reset_index()

# Function to summarize 5-day spans and 3-hour spans
def summarize_spans(df_grouped):
    day_spans = get_day_spans()
    results = []

    # Loop through all 5-day spans
    for span in day_spans:
        # Filter the data for these days
        span_data = df_grouped[df_grouped['DayOfWeek'].isin(span)]
        
        # Loop through 3-hour spans
        for start_hour in range(0, 22, 3):
            time_data = span_data[(span_data['Hour'] >= start_hour) & (span_data['Hour'] < start_hour + 3)]
            
            # Summarize key metrics
            total_visitors = time_data['ผู้เข้าชมสินค้า'].sum()
            total_sales = time_data['ยอดขาย (ยืนยันแล้ว) (THB)'].sum()
            avg_conversion_rate = time_data['จำนวนที่ขายได้ (ยืนยันแล้ว)'].sum() / total_visitors if total_visitors > 0 else 0

            # Append the result for this span
            results.append({
                'DaySpan': f"{span[0]} to {span[-1]}",
                'TimeSpan': f"{start_hour}:00 - {start_hour + 3}:00",
                'TotalVisitors': total_visitors,
                'TotalSales (THB)': total_sales,
                'AvgConversionRate': avg_conversion_rate
            })

    return pd.DataFrame(results)

# Summarize the spans and sort the results by the highest sales and conversion rate
summary_df = summarize_spans(df_grouped)
summary_df_sorted = summary_df.sort_values(by=['TotalSales (THB)', 'AvgConversionRate'], ascending=False)

# Display the top 10 results in the console
print(summary_df_sorted.head(10))

summary_df_sorted.to_excel('flash_sale_summary.xlsx', index=False)




                DaySpan       TimeSpan  TotalVisitors  TotalSales (THB)  \
0      Monday to Friday    0:00 - 3:00          490.0           34727.0   
19  Wednesday to Sunday   9:00 - 12:00          923.0           28491.0   
4      Monday to Friday  12:00 - 15:00         1005.0           28218.0   
7      Monday to Friday  21:00 - 24:00         1309.0           27752.0   
11  Tuesday to Saturday   9:00 - 12:00          906.0           27343.0   
12  Tuesday to Saturday  12:00 - 15:00          925.0           27252.0   
20  Wednesday to Sunday  12:00 - 15:00          929.0           26191.0   
23  Wednesday to Sunday  21:00 - 24:00         1421.0           22590.0   
3      Monday to Friday   9:00 - 12:00          959.0           22058.0   
15  Tuesday to Saturday  21:00 - 24:00         1246.0           20923.0   

    AvgConversionRate  
0            0.073469  
19           0.022752  
4            0.025871  
7            0.019099  
11           0.025386  
12           0.029189  
20    